# Beurteilung der Datenqualität

In [ ]:
%pip install eurostat

In [ ]:
import eurostat
df = eurostat.get_data_df("prc_hpi_a")
df

In [ ]:
hp10 = df[(df["unit"] == "I10_A_AVG") & (df["purchase"] == "DW_NEW")].copy()
hp10

In [ ]:
hp10.describe()

In [ ]:
hp10.dropna()

In [ ]:
columns = [str(c) for c in range(2010,2022)]
hp10.dropna(subset=columns)

In [ ]:
hp_from_2010 = hp10.dropna(subset=columns)[["geo\\TIME_PERIOD"] + columns]
hp_from_2010

In [ ]:
hp_from_2010.rename(columns={"geo\\TIME_PERIOD": "country"}, inplace=True)
hp10.rename(columns={"geo\\TIME_PERIOD": "country"}, inplace=True)

In [ ]:
hp10[hp10["country"] == "DE"]

In [ ]:
hp10[hp10["country"] == "DE"][[str(i) for i in range(2010, 2022)]].T.plot.bar()

In [ ]:
country_prices = hp10[["country"]+ [str(c) for c in range(2005,2022)]].set_index("country")
country_prices.T.plot(figsize=(16,9))

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
sns.heatmap(country_prices)

In [ ]:
sns.jointplot(x=country_prices.T["DE"], y=country_prices.T["AT"], scatter=False, kind="reg")

In [ ]:
sns.jointplot(x=country_prices.T["DE"], y=country_prices.T["TR"], scatter=False, kind="reg")

In [ ]:
country_prices.T[["DE", "AT", "TR"]].corr()

In [ ]:
sns.pairplot(country_prices.T[["DE", "AT", "TR"]])

In [ ]:
country_prices

In [ ]:
import pandas as pd
u = country_prices.melt(ignore_index=False, var_name="year", value_name="price_index").reset_index()

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 10))
sns.boxplot(y = u["country"], x = u["price_index"])